#Camada Silver

## Carregamento das tabelas da camada bronze

In [0]:
# Importação de bibliotecas
from pyspark.sql import functions as F
from pyspark.sql.window import Window
from datetime import timedelta


In [0]:
# Definição do caminho da pasta Silver
catalogo = "medalhao"
silver_db_name = "silver"

In [0]:
spark.sql("DROP TABLE IF EXISTS medalhao.silver.ft_consumidores")
spark.sql("DROP TABLE IF EXISTS medalhao.silver.ft_produtos")
spark.sql("DROP TABLE IF EXISTS medalhao.silver.ft_pedidos")
spark.sql("DROP TABLE IF EXISTS medalhao.silver.ft_vendedores") 
spark.sql("DROP TABLE IF EXISTS medalhao.silver.ft_avaliacoes_pedidos")
spark.sql("DROP TABLE IF EXISTS medalhao.silver.ft_itens_pedidos")
spark.sql("DROP TABLE IF EXISTS medalhao.silver.ft_pagamentos_pedidos")
spark.sql("DROP TABLE IF EXISTS medalhao.silver.ft_cotacao_dolar")
spark.sql("DROP TABLE IF EXISTS medalhao.silver.dm_categoria_produtos")
spark.sql("DROP TABLE IF EXISTS medalhao.silver.ft_pedido_total")


DataFrame[]

In [0]:
# Leitura das tabelas Bronze
dm_categoria_produtos_bronze_traducao_df = spark.table("medalhao.bronze.dm_categoria_produtos_traducao")
dm_cotacao_dolar_bronze_df = spark.table("medalhao.bronze.dm_cotacao_dolar")
ft_avaliacoes_pedidos_bronze_df = spark.table("medalhao.bronze.ft_avaliacoes_pedidos")
ft_consumidores_bronze_df = spark.table("medalhao.bronze.ft_consumidores")
ft_itens_pedidos_bronze_df = spark.table("medalhao.bronze.ft_itens_pedidos")
ft_pagamentos_pedidos_bronze_df = spark.table("medalhao.bronze.ft_pagamentos_pedidos")
ft_pedidos_bronze_df = spark.table("medalhao.bronze.ft_pedidos")
ft_produtos_bronze_df = spark.table("medalhao.bronze.ft_produtos")
ft_vendedores_bronze_df = spark.table("medalhao.bronze.ft_vendedores")
print("Tabelas Bronze carregadas com sucesso!\n")


Tabelas Bronze carregadas com sucesso!



## 1. ft_consumidores


### Etapas de transformação dos dados
- Renomear colunas para padronizar em português
- Remover duplicatas com base na coluna `id_consumidor` 
- Converter os nomes de estados e cidades para maiúsculo


In [0]:
ft_consumidores_bronze_df.printSchema()

root
 |-- customer_id: string (nullable = true)
 |-- customer_unique_id: string (nullable = true)
 |-- customer_zip_code_prefix: integer (nullable = true)
 |-- customer_city: string (nullable = true)
 |-- customer_state: string (nullable = true)
 |-- ingestion_timestamp: timestamp (nullable = true)



In [0]:
ft_consumidores_silver_df = (
    ft_consumidores_bronze_df
    .select(
        F.col("customer_id").alias("id_consumidor"),
        F.col("customer_zip_code_prefix").alias("prefixo_cep"),
        F.upper(F.col("customer_city")).alias("cidade"),
        F.upper(F.col("customer_state")).alias("estado")
    )
    .filter(F.col("id_consumidor").isNotNull())
    .dropDuplicates(["id_consumidor"])
    .withColumn("ingestion_timestamp", F.current_timestamp())
)


ft_consumidores_silver_df.write.format("delta").mode("overwrite").saveAsTable(f"{catalogo}.{silver_db_name}.ft_consumidores")
display(ft_consumidores_silver_df.limit(5))

id_consumidor,prefixo_cep,cidade,estado,ingestion_timestamp
06b8999e2fba1a1fbc88172c00ba8bc7,14409,FRANCA,SP,2025-11-18T21:41:19.174Z
18955e83d337fd6b2def6b18a428ac77,9790,SAO BERNARDO DO CAMPO,SP,2025-11-18T21:41:19.174Z
4e7b3e00288586ebd08712fdd0374a03,1151,SAO PAULO,SP,2025-11-18T21:41:19.174Z
b2b6027bc5c5109e529d4dc6358b12c3,8775,MOGI DAS CRUZES,SP,2025-11-18T21:41:19.174Z
4f2d8ab171c80ec8364f7c12e35b23ad,13056,CAMPINAS,SP,2025-11-18T21:41:19.174Z


## 2. ft_pedidos

### Etapas de transformação dos dados
- Renomear colunas para padronizar em português
- Converter valores da coluna `status` para português
- Criar novas colunas: tempo_entrega_dias, tempo_entrega_estimado_dias, diferenca_entrega_dias,
entrega_no_prazo

In [0]:
ft_pedidos_bronze_df.printSchema()

root
 |-- order_id: string (nullable = true)
 |-- customer_id: string (nullable = true)
 |-- order_status: string (nullable = true)
 |-- order_purchase_timestamp: timestamp (nullable = true)
 |-- order_approved_at: timestamp (nullable = true)
 |-- order_delivered_carrier_date: timestamp (nullable = true)
 |-- order_delivered_customer_date: timestamp (nullable = true)
 |-- order_estimated_delivery_date: timestamp (nullable = true)
 |-- ingestion_timestamp: timestamp (nullable = true)



In [0]:
# Mapeamento para transformação da coluna order_status
mapeamento_pedidos = {
    "delivered": "entregue",
    "invoiced": "faturado",
    "shipped": "enviado",
    "processing": "em processamento",
    "unavailable": "indisponível",
    "canceled": "cancelado",
    "created": "criado",
    "approved": "aprovado"
}

In [0]:

# Criar o DataFrame Silver com o status traduzido
ft_pedidos_silver_df = (
    ft_pedidos_bronze_df
    .select(
        F.col("order_id").alias("id_pedido"),
        F.col("customer_id").alias("id_consumidor"),
        F.col("order_status").alias("status"),
        F.col("order_purchase_timestamp").alias("pedido_compra_timestamp"),
        F.col("order_approved_at").alias("pedido_aprovado_timestamp"),
        F.col("order_delivered_carrier_date").alias("pedido_carregado_timestamp"),
        F.col("order_delivered_customer_date").alias("pedido_entregue_timestamp"),
        F.col("order_estimated_delivery_date").alias("pedido_estimativa_entrega_timestamp")
    )
    .withColumn(
        "status",
        F.create_map([F.lit(x) for x in sum(mapeamento_pedidos.items(), ())])[F.col("status")]
    )
    .withColumn("tempo_entrega_dias", F.datediff(F.col("pedido_entregue_timestamp"), F.col("pedido_compra_timestamp")))
    .withColumn("tempo_entrega_estimado_dias", F.datediff(F.col("pedido_estimativa_entrega_timestamp"), F.col("pedido_compra_timestamp")))
    .withColumn("diferenca_entrega_dias", F.datediff(F.col("pedido_entregue_timestamp"), F.col("pedido_estimativa_entrega_timestamp")))
    .withColumn("entrega_no_prazo", F.when(F.col("diferenca_entrega_dias") <= 0, "Sim")
                                    .when(F.col("status") != "entregue", "Não Entregue")
                                    .otherwise("Não"))
    .withColumn("ingestion_timestamp", F.current_timestamp())


)
ft_pedidos_silver_df.write.format("delta").mode("overwrite").saveAsTable(f"{catalogo}.{silver_db_name}.ft_pedidos")
display(ft_pedidos_silver_df.limit(5))


id_pedido,id_consumidor,status,pedido_compra_timestamp,pedido_aprovado_timestamp,pedido_carregado_timestamp,pedido_entregue_timestamp,pedido_estimativa_entrega_timestamp,tempo_entrega_dias,tempo_entrega_estimado_dias,diferenca_entrega_dias,entrega_no_prazo,ingestion_timestamp
e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,entregue,2017-10-02T10:56:33.000Z,2017-10-02T11:07:15.000Z,2017-10-04T19:55:00.000Z,2017-10-10T21:25:13.000Z,2017-10-18T00:00:00.000Z,8,16,-8,Sim,2025-11-18T21:41:39.462Z
53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,entregue,2018-07-24T20:41:37.000Z,2018-07-26T03:24:27.000Z,2018-07-26T14:31:00.000Z,2018-08-07T15:27:45.000Z,2018-08-13T00:00:00.000Z,14,20,-6,Sim,2025-11-18T21:41:39.462Z
47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,entregue,2018-08-08T08:38:49.000Z,2018-08-08T08:55:23.000Z,2018-08-08T13:50:00.000Z,2018-08-17T18:06:29.000Z,2018-09-04T00:00:00.000Z,9,27,-18,Sim,2025-11-18T21:41:39.462Z
949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,entregue,2017-11-18T19:28:06.000Z,2017-11-18T19:45:59.000Z,2017-11-22T13:39:59.000Z,2017-12-02T00:28:42.000Z,2017-12-15T00:00:00.000Z,14,27,-13,Sim,2025-11-18T21:41:39.462Z
ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,entregue,2018-02-13T21:18:39.000Z,2018-02-13T22:20:29.000Z,2018-02-14T19:46:34.000Z,2018-02-16T18:17:02.000Z,2018-02-26T00:00:00.000Z,3,13,-10,Sim,2025-11-18T21:41:39.462Z


## 3. ft_itens_pedidos

### Etapas de transformação dos dados
- Renomear colunas para padronizar em português
- Transformar tipo de `preco_BRL` e `preco_frete`

In [0]:
ft_itens_pedidos_bronze_df.printSchema()

root
 |-- order_id: string (nullable = true)
 |-- order_item_id: integer (nullable = true)
 |-- product_id: string (nullable = true)
 |-- seller_id: string (nullable = true)
 |-- shipping_limit_date: timestamp (nullable = true)
 |-- price: double (nullable = true)
 |-- freight_value: double (nullable = true)
 |-- ingestion_timestamp: timestamp (nullable = true)



In [0]:
ft_itens_pedidos_silver_df = (
    ft_itens_pedidos_bronze_df
    .select(
        F.col("order_id").alias("id_pedido"),
        F.col("order_item_id").alias("id_item"),
        F.col("product_id").alias("id_produto"),
        F.col("seller_id").alias("id_vendedor"),
        F.col("price").alias("preco_BRL").cast("decimal(10,2)"),
        F.col("freight_value").alias("preco_frete").cast("decimal(10,2)")
    )
    .withColumn("ingestion_timestamp", F.current_timestamp())
)
ft_itens_pedidos_silver_df.write.format("delta").mode("overwrite").saveAsTable(f"{catalogo}.{silver_db_name}.ft_itens_pedidos")
display(ft_itens_pedidos_silver_df.limit(5))

id_pedido,id_item,id_produto,id_vendedor,preco_BRL,preco_frete,ingestion_timestamp
00010242fe8c5a6d1ba2dd792cb16214,1,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,58.90,13.29,2025-11-18T21:41:44.519Z
00018f77f2f0320c557190d7a144bdd3,1,e5f2d52b802189ee658865ca93d83a8f,dd7ddc04e1b6c2c614352b383efe2d36,239.90,19.93,2025-11-18T21:41:44.519Z
000229ec398224ef6ca0657da4fc703e,1,c777355d18b72b67abbeef9df44fd0fd,5b51032eddd242adc84c38acab88f23d,199.00,17.87,2025-11-18T21:41:44.519Z
00024acbcdf0a6daa1e931b038114c75,1,7634da152a4610f1595efa32f14722fc,9d7a1d34a5052409006425275ba1c2b4,12.99,12.79,2025-11-18T21:41:44.519Z
00042b26cf59d7ce69dfabb4e55b4fd9,1,ac6c3623068f30de03045865e4e10089,df560393f3a51e74553ab94004ba5c87,199.90,18.14,2025-11-18T21:41:44.519Z


## 4. ft_pagamentos


### Etapas de transformação dos dados
- Renomear colunas para padronizar em português
- Converter valores da coluna `forma_pagamento` para português
- Transformar tipo de `valor_pagamento`


In [0]:
ft_pagamentos_pedidos_bronze_df.printSchema()

root
 |-- order_id: string (nullable = true)
 |-- payment_sequential: integer (nullable = true)
 |-- payment_type: string (nullable = true)
 |-- payment_installments: integer (nullable = true)
 |-- payment_value: double (nullable = true)
 |-- ingestion_timestamp: timestamp (nullable = true)



In [0]:
mapeamento_pagamentos = {
    "boleto": "Boleto",
    "credit_card": "Cartão de Crédito",
    "debit_card": "Cartão de Débito",
    "voucher": "Voucher"
}

In [0]:
ft_pagamentos_pedidos_silver_df = (
    ft_pagamentos_pedidos_bronze_df
    .select(
        F.col("order_id").alias("id_pedido"),
        F.col("payment_sequential").alias("codigo_pagamento"),
        F.col("payment_type").alias("forma_pagamento"),
        F.col("payment_installments").alias("parcelas"),
        F.col("payment_value").alias("valor_pagamento").cast("decimal(10,2)")
    )
    .withColumn(
        "forma_pagamento",
        F.coalesce( 
            F.create_map([F.lit(x) for x in sum(mapeamento_pagamentos.items(), ())])[F.col("forma_pagamento")],
            F.lit("Outro")  
        )
    )
)
    
ft_pagamentos_pedidos_silver_df.write.format("delta").mode("overwrite").saveAsTable(f"{catalogo}.{silver_db_name}.ft_pagamentos_pedidos")
display(ft_pagamentos_pedidos_silver_df.limit(5))

id_pedido,codigo_pagamento,forma_pagamento,parcelas,valor_pagamento
b81ef226f3fe1789b1e8b2acac839d17,1,Cartão de Crédito,8,99.33
a9810da82917af2d9aefd1278f1dcfa0,1,Cartão de Crédito,1,24.39
25e8ea4e93396b6fa0d3dd708e76c1bd,1,Cartão de Crédito,1,65.71
ba78997921bbcdc1373bb41e913ab953,1,Cartão de Crédito,8,107.78
42fdf880ba16b47b59251dd489d4441a,1,Cartão de Crédito,2,128.45


## 5. ft_avaliacoes_pedidos


### Etapas de transformação dos dados
- Renomear colunas para padronizar em português
- Remover registros com `id_pedido` inválido ou datas incorretas (ex.: data nula, formato
inconsistente, data futura fora do escopo)
- Tranformar tipo de `avaliacao`, `data_comentario` e `data_resposta`



In [0]:
ft_avaliacoes_pedidos_bronze_df.printSchema()

root
 |-- review_id: string (nullable = true)
 |-- order_id: string (nullable = true)
 |-- review_score: string (nullable = true)
 |-- review_comment_title: string (nullable = true)
 |-- review_comment_message: string (nullable = true)
 |-- review_creation_date: string (nullable = true)
 |-- review_answer_timestamp: string (nullable = true)
 |-- ingestion_timestamp: timestamp (nullable = true)



**Requisitos para id_pedido válido:**
- Não possuir espaços
- Somente letras minúsculas e números
- Não deve ser uma data
- Não ser nulo

**Requisitos para datas válidas:**
- Data não nula
- Data formato adequado
- Data de comentário anterior a data de resposta
- Data anterior ao timestamp atual

In [0]:
ft_avaliacoes_pedidos_silver_df = (
    ft_avaliacoes_pedidos_bronze_df
    .select(
        F.col("review_id").alias("id_avaliacao"),
        F.col("order_id").alias("id_pedido"),
        F.col("review_score").alias("avaliacao").cast("integer"),
        F.col("review_comment_title").alias("titulo_comentario"),
        F.col("review_comment_message").alias("comentario"),
        F.col("review_creation_date").alias("data_comentario"),
        F.col("review_answer_timestamp").alias("data_resposta")
    )
    # Filtra apenas id_pedido válidos
    .filter(
        (F.col("id_pedido").isNotNull()) &  # Remove nulos
        (~F.col("id_pedido").rlike(r"\s")) &  # Não pode ter espaços
        (F.col("id_pedido").rlike(r"^[a-z0-9]+$")) &  # Apenas letras minúsculas e números
        (~F.col("id_pedido").rlike(r"^\d{4}-\d{2}-\d{2}$")) # Não deve ser uma data
    )
    # Converte as colunas de data para timestamp, quando não é possível converter, substitui por nulo
    .withColumn("data_comentario", F.try_to_timestamp("data_comentario"))
    .withColumn("data_resposta", F.try_to_timestamp("data_resposta"))
    .filter(
        (F.col("data_comentario").isNotNull()) & # Remove nulos
        (F.col("data_resposta").isNotNull()) &
        (F.col("data_comentario") < F.col("data_resposta")) & # Datas de resposta devem ser posteriores às datas de comentário
        (F.col("data_comentario") < F.current_timestamp()) & # Datas de comentário devem ser anteriores à atual
        (F.col("data_resposta") < F.current_timestamp()) # Datas de resposta devem ser anteriores à atual
    )
    .withColumn("ingestion_timestamp", F.current_timestamp())
)


ft_avaliacoes_pedidos_silver_df.write.format("delta").mode("overwrite").saveAsTable(f"{catalogo}.{silver_db_name}.ft_avaliacoes_pedidos")
display(ft_avaliacoes_pedidos_silver_df.limit(5))

id_avaliacao,id_pedido,avaliacao,titulo_comentario,comentario,data_comentario,data_resposta,ingestion_timestamp
7bc2406110b926393aa56f80a40eba40,73fc7af87114b39712e6da79b0a377eb,4,null,null,2018-01-18T00:00:00.000Z,2018-01-18T21:46:59.000Z,2025-11-18T21:41:53.639Z
80e641a11e56f04c1ad469d5645fdfde,a548910a1c6147796b98fdf73dbeba33,5,null,null,2018-03-10T00:00:00.000Z,2018-03-11T03:05:13.000Z,2025-11-18T21:41:53.639Z
228ce5500dc1d8e020d8d1322874b6f0,f9e4b658b201a9f2ecdecbb34bed034b,5,null,null,2018-02-17T00:00:00.000Z,2018-02-18T14:36:24.000Z,2025-11-18T21:41:53.639Z
e64fb393e7b32834bb789ff8bb30750e,658677c97b385a9be170737859d3511b,5,null,Recebi bem antes do prazo estipulado.,2017-04-21T00:00:00.000Z,2017-04-21T22:02:06.000Z,2025-11-18T21:41:53.639Z
f7c4243c7fe1938f181bec41a392bdeb,8e6bfb81e283fa7e4f11123a3fb894f1,5,null,Parabéns lojas lannister adorei comprar pela Internet seguro e prático Parabéns a todos feliz Páscoa,2018-03-01T00:00:00.000Z,2018-03-02T10:26:53.000Z,2025-11-18T21:41:53.639Z


## 6. ft_produtos

### Etapas de transformação dos dados
- Renomear colunas para padronizar em português

In [0]:
ft_produtos_bronze_df.printSchema()

root
 |-- product_id: string (nullable = true)
 |-- product_category_name: string (nullable = true)
 |-- product_name_lenght: integer (nullable = true)
 |-- product_description_lenght: integer (nullable = true)
 |-- product_photos_qty: integer (nullable = true)
 |-- product_weight_g: integer (nullable = true)
 |-- product_length_cm: integer (nullable = true)
 |-- product_height_cm: integer (nullable = true)
 |-- product_width_cm: integer (nullable = true)
 |-- ingestion_timestamp: timestamp (nullable = true)



In [0]:
ft_produtos_silver_df = (
    ft_produtos_bronze_df
    .select(
        F.col("product_id").alias("id_produto"),
        F.col("product_category_name").alias("categoria_produto"),
        F.col("product_weight_g").alias("peso_produto_gramas"),
        F.col("product_length_cm").alias("comprimento_centimetros"),
        F.col("product_height_cm").alias("altura_centimetros"),
        F.col("product_width_cm").alias("largura_centimetros")
    )
    .withColumn("ingestion_timestamp", F.current_timestamp())
)

ft_produtos_silver_df.write.format("delta").mode("overwrite").saveAsTable(f"{catalogo}.{silver_db_name}.ft_produtos")
display(ft_produtos_silver_df.limit(5))

id_produto,categoria_produto,peso_produto_gramas,comprimento_centimetros,altura_centimetros,largura_centimetros,ingestion_timestamp
1e9e8ef04dbcff4541ed26657ea517e5,perfumaria,225,16,10,14,2025-11-18T21:41:57.678Z
3aa071139cb16b67ca9e5dea641aaa2f,artes,1000,30,18,20,2025-11-18T21:41:57.678Z
96bd76ec8810374ed1b65e291975717f,esporte_lazer,154,18,9,15,2025-11-18T21:41:57.678Z
cef67bcfe19066a932b7673e239eb23d,bebes,371,26,4,26,2025-11-18T21:41:57.678Z
9dc1a7de274444849c219cff195d0b71,utilidades_domesticas,625,20,17,13,2025-11-18T21:41:57.678Z


## 7. ft_vendedores

### Etapas de transformação dos dados
- Renomear colunas para padronizar em português
- Converter os nomes de estados e cidades para maiúsculo

In [0]:
ft_vendedores_bronze_df.printSchema()

root
 |-- seller_id: string (nullable = true)
 |-- seller_zip_code_prefix: integer (nullable = true)
 |-- seller_city: string (nullable = true)
 |-- seller_state: string (nullable = true)
 |-- ingestion_timestamp: timestamp (nullable = true)



In [0]:
ft_vendedores_silver_df = (
    ft_vendedores_bronze_df
    .select(
        F.col("seller_id").alias("id_vendedor"),
        F.col("seller_zip_code_prefix").alias("prefixo_cep"),
        F.upper(F.col("seller_city")).alias("cidade"),
        F.upper(F.col("seller_state")).alias("estado")
    )
    .withColumn("ingestion_timestamp", F.current_timestamp())
)
    
ft_vendedores_silver_df.write.format("delta").mode("overwrite").saveAsTable(f"{catalogo}.{silver_db_name}.ft_vendedores")
display(ft_vendedores_silver_df.limit(5))

id_vendedor,prefixo_cep,cidade,estado,ingestion_timestamp
3442f8959a84dea7ee197c632cb2df15,13023,CAMPINAS,SP,2025-11-18T21:42:01.714Z
d1b65fc7debc3361ea86b5f14c68d2e2,13844,MOGI GUACU,SP,2025-11-18T21:42:01.714Z
ce3ad9de960102d0677a81f5d0bb7b2d,20031,RIO DE JANEIRO,RJ,2025-11-18T21:42:01.714Z
c0f3eea2e14555b6faeea3dd58c1b1c3,4195,SAO PAULO,SP,2025-11-18T21:42:01.714Z
51a04a8a6bdcb23deccc82b0b80742cf,12914,BRAGANCA PAULISTA,SP,2025-11-18T21:42:01.714Z


## 8. dm_categoria_produtos_traducao


### Etapas de transformação dos dados
- Renomear colunas para padronizar em português

In [0]:
dm_categoria_produtos_bronze_traducao_df.printSchema()

root
 |-- product_category_name: string (nullable = true)
 |-- product_category_name_english: string (nullable = true)
 |-- ingestion_timestamp: timestamp (nullable = true)



In [0]:
dm_categoria_produtos_silver_traducao_df = (
    dm_categoria_produtos_bronze_traducao_df
    .select(
        F.col("product_category_name").alias("nome_produto_pt"),
        F.col("product_category_name_english").alias("nome_produto_en")
    )
    .withColumn("ingestion_timestamp", F.current_timestamp())
)

dm_categoria_produtos_silver_traducao_df.write.format("delta").mode("overwrite").saveAsTable(f"{catalogo}.{silver_db_name}.dm_categoria_produtos_traducao")
display(dm_categoria_produtos_silver_traducao_df.limit(5))

nome_produto_pt,nome_produto_en,ingestion_timestamp
beleza_saude,health_beauty,2025-11-18T21:42:05.057Z
informatica_acessorios,computers_accessories,2025-11-18T21:42:05.057Z
automotivo,auto,2025-11-18T21:42:05.057Z
cama_mesa_banho,bed_bath_table,2025-11-18T21:42:05.057Z
moveis_decoracao,furniture_decor,2025-11-18T21:42:05.057Z


## 9. dm_cotacao_dolar

### Etapas de transformação dos dados
- Renomear colunas 
- Substituir cotação do fim de semana por cotação de fechamento da sexta-feira


In [0]:
dm_cotacao_dolar_bronze_df.printSchema()

root
 |-- cotacaoCompra: double (nullable = true)
 |-- cotacaoVenda: double (nullable = true)
 |-- dataHoraCotacao: string (nullable = true)
 |-- ingestion_timestamp: timestamp (nullable = true)



In [0]:
dm_cotacao_dolar_silver_df = (
    dm_cotacao_dolar_bronze_df
    .select(
        F.col("cotacaoCompra").alias("cotacao_dolar"),
        F.to_timestamp(F.col("dataHoraCotacao"), "yyyy-MM-dd HH:mm:ss.SSS").alias("data")
    )
)

# Pega intervalo de datas do DataFrame
min_data = dm_cotacao_dolar_silver_df.select(F.min("data")).collect()[0][0].date()
max_data = dm_cotacao_dolar_silver_df.select(F.max("data")).collect()[0][0].date()

# cria um DataFrame com todas as datas do período e filtra as do fim de semana
datas_fim_de_semana = (
    spark.createDataFrame([(1,)], ["dummy"])  # seed para usar sequence/explode
    .select(
        F.explode(
            F.sequence(F.to_date(F.lit(min_data)), F.to_date(F.lit(max_data)), F.expr("interval 1 day"))
        ).alias("date_only")
    )
    .withColumn("data", F.to_timestamp("date_only"))  # transforma em timestamp 
    .select("data")
    .filter((F.dayofweek(F.col("data")) == 1) | (F.dayofweek(F.col("data")) == 7))  # mantém somente datas de domingo e sábado

)

# Janela para o preenchimento
w = Window.orderBy("data").rowsBetween(Window.unboundedPreceding, 0)

# Mantém todas as datas de ambos os DataFrames
juncao_datas = (
    datas_fim_de_semana
    .join(dm_cotacao_dolar_silver_df, on="data", how="outer")
)

# Faz o preenchimento
dm_cotacao_dolar_silver_df = (
    juncao_datas
    .withColumn(
        "cotacao_dolar",
        F.when(
            F.dayofweek("data").isin([1, 7]),  # Sábado ou domingo
            F.last("cotacao_dolar", ignorenulls=True).over(w)  # Usa última cotação disponível
        ).otherwise(F.col("cotacao_dolar"))  # Mantém cotação do próprio dia
    )
    .withColumn("ingestion_timestamp", F.current_timestamp())
)

dm_cotacao_dolar_silver_df.write.format("delta").mode("overwrite").saveAsTable(f"{catalogo}.{silver_db_name}.dm_cotacao_dolar")
display(dm_cotacao_dolar_silver_df.limit(10))
    

/databricks/python/lib/python3.12/site-packages/pyspark/sql/connect/expressions.py:1134: UserWarning: WARN WindowExpression: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
  warnings.warn(
/databricks/python/lib/python3.12/site-packages/pyspark/sql/connect/expressions.py:1134: UserWarning: WARN WindowExpression: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
  warnings.warn(


data,cotacao_dolar,ingestion_timestamp
2016-09-05T13:09:55.659Z,3.2715,2025-11-18T21:42:09.359Z
2016-09-06T13:02:39.984Z,3.2446,2025-11-18T21:42:09.359Z
2016-09-08T13:03:53.968Z,3.1928,2025-11-18T21:42:09.359Z
2016-09-09T13:14:00.885Z,3.2632,2025-11-18T21:42:09.359Z
2016-09-10T00:00:00.000Z,3.2632,2025-11-18T21:42:09.359Z
2016-09-11T00:00:00.000Z,3.2632,2025-11-18T21:42:09.359Z
2016-09-12T13:08:01.541Z,3.2848,2025-11-18T21:42:09.359Z
2016-09-13T13:03:56.534Z,3.2966,2025-11-18T21:42:09.359Z
2016-09-14T13:05:51.819Z,3.3256,2025-11-18T21:42:09.359Z
2016-09-15T13:08:34.825Z,3.332,2025-11-18T21:42:09.359Z


## Verificação de integridade referencial 

- Todos os pedidos possuam um consumidor válido (ou seja, não existam pedidos órfãos sem
correspondência na tabela de consumidores)
- Todos os itens de pedidos estejam associados a um pedido existente

In [0]:
# Identifica pedidos órfãos de consumidores
pedidos_orfaos_df = ft_pedidos_silver_df.join(
    ft_consumidores_silver_df,
    on="id_consumidor",
    how="left_anti"
)

# Conta quantos pedidos órfãos existem
qtd_pedidos_orfaos = pedidos_orfaos_df.count()

print(f"Quantidade de pedidos órfãos: {qtd_pedidos_orfaos}")

Quantidade de pedidos órfãos: 0


In [0]:
# Identifica itens órfãos de pedido
itens_orfaos_df = ft_itens_pedidos_silver_df.join(
    ft_pedidos_silver_df,
    on="id_pedido",
    how="left_anti"
)

# Conta quantos itens órfãos existem
qtd_itens_orfaos = itens_orfaos_df.count()

print(f"Quantidade de itens órfãos: {qtd_itens_orfaos}")


Quantidade de itens órfãos: 0


## Criação da tabela silver.ft_pedido_total

**Junção das tabelas:**
- ft_pedidos 
- ft_consumidores
- ft_pagamentos_pedidos 
- dm_cotacao_dolar 




**É importante salientar que alguns valores em valor_total_pago_usd serão nulos, pois não há cotação em dias de feriado, como dia 2017-06-15 (Corpus Christi)**

In [0]:
# Para definir o intervalo desejado para a cotação do dolar
min_ts = ft_pedidos_silver_df.select(F.min("pedido_compra_timestamp")).collect()[0][0]
max_ts = ft_pedidos_silver_df.select(F.max("pedido_compra_timestamp")).collect()[0][0]

print("Mínimo:", min_ts)
print("Máximo:", max_ts)

Mínimo: 2016-09-04 21:15:19
Máximo: 2018-10-17 17:30:18


In [0]:
# Junção de pedidos e consumidores
pedidos_consumidores_df = (
    ft_pedidos_bronze_df
    .join(
        ft_consumidores_bronze_df,
        on="customer_id",
        how="inner"
    )
    .select(
        F.col("order_id").alias("id_pedido"),
        F.col("customer_id").alias("id_consumidor"),
        F.col("order_status").alias("status"),
        F.to_timestamp("order_purchase_timestamp").alias("data_pedido")
    )
)

# Junção com pagamentos (somando valor total por pedido)
pedidos_consumidores_pagamentos_df = (
    pedidos_consumidores_df
    .join(
        ft_pagamentos_pedidos_bronze_df,
        pedidos_consumidores_df.id_pedido == ft_pagamentos_pedidos_bronze_df.order_id,
        how="inner"
    )
    .groupBy(
        "id_pedido", "id_consumidor", "status", "data_pedido"
    )
    .agg(
        F.sum("payment_value").alias("valor_total_pago_brl").cast("Decimal(10,2)")
    )
)

# Converter data para tipo date para realizar o join
pedidos_consumidores_pagamentos_df = pedidos_consumidores_pagamentos_df.withColumn(
    "data_pedido", F.to_date("data_pedido")
)
 
dm_cotacao_dolar_df = dm_cotacao_dolar_silver_df.withColumn(
    "data", F.to_date("data")
)

# Join com cotação do dólar
df_final = (
    pedidos_consumidores_pagamentos_df
    .join(
        dm_cotacao_dolar_df.select("data", "cotacao_dolar"),
        pedidos_consumidores_pagamentos_df.data_pedido == dm_cotacao_dolar_df.data,
        how="left"
    )
    .withColumn(
        "valor_total_pago_usd",
        F.when(
            (F.col("cotacao_dolar").isNull()) | (F.col("cotacao_dolar") == 0),
            None
        ).otherwise(
            F.col("valor_total_pago_brl") / F.col("cotacao_dolar")
        ).cast("Decimal(10,2)")
    )
    .select(
        "id_pedido",
        "id_consumidor",
        "status",
        "valor_total_pago_brl",
        "valor_total_pago_usd",
        "data_pedido"
    )
)
df_final.write.format("delta").mode("overwrite").saveAsTable(f"{catalogo}.{silver_db_name}.ft_pedido_total")
display(df_final.limit(10))


id_pedido,id_consumidor,status,valor_total_pago_brl,valor_total_pago_usd,data_pedido
f4471dae8c482f51aa1826cd9f5d4433,167b9485947ed0a354a3f8dad04eb199,delivered,232.14,59.24,2018-07-05
377627ea55b004e8d55a2af17ba3c0e1,3f0bd16414d5fee5cb1d19769c3c825e,delivered,207.26,55.27,2018-05-20
36da3594f270f9d28bae07df5974d240,4d1e62429d88723ea3c9a72fb4c4ffe3,delivered,164.54,51.70,2017-10-21
67009d1f213f762960c97b4f8629159c,d3e3cb2b2922f53b23b090d9668a1b13,delivered,216.83,67.59,2018-01-20
0ac372f92fbec55ca27ef4fa99a6a13a,96d3328d8b8625bc4d656279e1515bee,delivered,23.10,7.15,2018-01-11
a11f0312591acf976fd9bf40d6b539db,007b18ac9b8a627f259ea78aed981315,delivered,43.13,13.11,2018-03-16
8d39667dd1c683e4a8b9dc1f16d08cb9,d3d640e9dd196770cd7e5c848e8a8152,delivered,641.82,203.58,2017-07-19
0c892a79e462e31570b88739be67f32d,946f35bd37b2438f7f2d68da8ed3b963,delivered,761.82,243.58,2017-03-24
314fc0121ed16cbe4f7979e96f3768ef,9b1c0b706b8991152859ed64739367f8,delivered,213.95,61.73,2018-04-24
9af5f003f0da353f6c32a5c5e0c83b39,2129cc3d2764d1f9c4618f6c06f40f65,delivered,353.31,111.74,2018-01-31
